In [ ]:
from monai.utils import set_determinism, first
from monai.transforms import (
    EnsureChannelFirstD,
    Compose,
    LoadImageD,
    RandRotateD,
    RandZoomD,
    ScaleIntensityRanged,
)
import monai
from monai.data import DataLoader, Dataset, CacheDataset
from monai.config import print_config, USE_COMPILED
from monai.networks.nets import GlobalNet, LocalNet, RegUNet
from monai.networks.blocks import Warp
from monai.apps import MedNISTDataset
import torch.nn.functional as F

from glob import glob
import cv2
import torchmetrics

from torch.autograd import Variable

from scipy.spatial.distance import directed_hausdorff
import pandas as pd

import torch.nn as nn

import numpy as np
import torch
from torch.nn import MSELoss
import matplotlib.pyplot as plt
import os
import tempfile
from monai.losses import *
from monai.metrics import *
from piqa import SSIM


print_config()
set_determinism(42)

In [ ]:
dataDir = 'CAMUS_EStoED_A2C'

root_dir = 'data/'+dataDir+'/'
print(root_dir)

fileNames = "adv_AC_DLIR_UNet" + dataDir

trainBatch = 8
testBatch = 8

img_size = 256

EP = 200

num_workers = 0

In [ ]:
print('How many GPUs = ' + str(torch.cuda.device_count()))

#checking for device

device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

if not torch.cuda.is_available():
  raise Exception("GPU not availalbe. CPU training will be too slow.")

print("device name", torch.cuda.get_device_name(0))

In [ ]:
class EchoDataset(Dataset):
    def __init__(self, images_path):

        self.images_path = images_path
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(img_size, img_size))
        image = image/(image.max()) ## (512, 512, 3)
#         print(image.max())
        image = np.expand_dims(image, axis=0)
        image = image.astype(np.float32)
        self.images_path[index]
        return image

    def __len__(self):
        return self.n_samples
    
class EchoDatasetMask(Dataset):
    def __init__(self, images_path):

        self.images_path = images_path
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image,(img_size, img_size), interpolation=cv2.INTER_NEAREST)
#         print(image.max())
#         image = image/(image.max()) ## (512, 512, 3)
        image = np.expand_dims(image, axis=0)
        image = image.astype(np.float32)
        self.images_path[index]
        return image

    def __len__(self):
        return self.n_samples

In [ ]:
def get_batches(train_dir,
                batch_size,
                num_workers,
                pin_memory):
    
    train_data = EchoDataset(images_path=train_dir)

    train_loader = DataLoader(train_data,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              pin_memory=pin_memory,
                              shuffle=False)

    return train_loader

def get_batches_mask(train_dir,
                batch_size,
                num_workers,
                pin_memory):
    
    train_data = EchoDatasetMask(images_path=train_dir)

    train_loader = DataLoader(train_data,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              pin_memory=pin_memory,
                              shuffle=False)

    return train_loader


print(f'Train Sample numbers (fixed_img) = {len(sorted(glob(root_dir+"/train/fixed_img/*.png")))}')
print(f'Train Sample numbers (fixed_msk) = {len(sorted(glob(root_dir+"/train/fixed_msk/*.png")))}')
print(f'Train Sample numbers (moving_img) = {len(sorted(glob(root_dir+"/train/moving_img/*.png")))}')
print(f'Train Sample numbers (moving_msk) = {len(sorted(glob(root_dir+"/train/moving_msk/*.png")))}')
print()
print(f'Val Sample numbers (fixed_img) = {len(sorted(glob(root_dir+"/val/fixed_img/*.png")))}')
print(f'Val Sample numbers (fixed_msk) = {len(sorted(glob(root_dir+"/val/fixed_msk/*.png")))}')
print(f'Val Sample numbers (moving_img) = {len(sorted(glob(root_dir+"/val/moving_img/*.png")))}')
print(f'Val Sample numbers (moving_msk) = {len(sorted(glob(root_dir+"/val/moving_msk/*.png")))}')
print()


fixed_train_img = get_batches(train_dir = sorted(glob(root_dir+"/train/fixed_img/*")),
                                        batch_size = trainBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)

fixed_train_msk = get_batches_mask(train_dir = sorted(glob(root_dir+"/train/fixed_msk/*")),
                                        batch_size = trainBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)


moving_train_img = get_batches(train_dir = sorted(glob(root_dir+"/train/moving_img/*")),
                                        batch_size = trainBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)

moving_train_msk = get_batches_mask(train_dir = sorted(glob(root_dir+"/train/moving_msk/*")),
                                        batch_size = trainBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)


print("Train IMG FIXED:", fixed_train_img)
print("Train MSK FIXED:", fixed_train_msk)
print("Train IMG Moving:", moving_train_img)
print("Train MSK Moving:", moving_train_msk)


fixed_val_img = get_batches(train_dir = sorted(glob(root_dir+"/val/fixed_img/*")),
                                        batch_size = testBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)

fixed_val_msk = get_batches_mask(train_dir = sorted(glob(root_dir+"/val/fixed_msk/*")),
                                        batch_size = testBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)


moving_val_img = get_batches(train_dir = sorted(glob(root_dir+"/val/moving_img/*")),
                                        batch_size = testBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)

moving_val_msk = get_batches_mask(train_dir = sorted(glob(root_dir+"/val/moving_msk/*")),
                                        batch_size = testBatch,
                                        num_workers = num_workers,
                                        pin_memory = True)


print("Val IMG FIXED:", fixed_val_img)
print("Val MSK FIXED:", fixed_val_msk)
print("Val IMG Moving:", moving_val_img)
print("Val MSK Moving:", moving_val_msk)

In [ ]:
dataloaders = {
      'fixed_train_img': fixed_train_img,
      'fixed_train_msk': fixed_train_msk,
    'moving_train_img': moving_train_img,
    'moving_train_msk': moving_train_msk,
    'fixed_val_img': fixed_val_img,
    'fixed_val_msk': fixed_val_msk,
    'moving_val_img': moving_val_img,
    'moving_val_msk': moving_val_msk
    }

In [ ]:
fixed_train_img_ = first(dataloaders["fixed_train_img"])[0][0]
fixed_train_msk_ = first(dataloaders["fixed_train_msk"])[0][0]
moving_train_img_ = first(dataloaders["moving_train_img"])[0][0]
moving_train_msk_ = first(dataloaders["moving_train_msk"])[0][0]

fixed_val_img_ = first(dataloaders["fixed_val_img"])[0][0]
fixed_val_msk_ = first(dataloaders["fixed_val_msk"])[0][0]
moving_val_img_ = first(dataloaders["moving_val_img"])[0][0]
moving_val_msk_ = first(dataloaders["moving_val_msk"])[0][0]


print(f"fixed_train_img_ shape: {fixed_train_img_.shape}")
print(f"fixed_train_msk_ shape: {fixed_train_msk_.shape}")
print(f"moving_train_img_ shape: {moving_train_img_.shape}")
print(f"moving_train_msk_ shape: {moving_train_msk_.shape}")

print(f"fixed_val_img_ shape: {fixed_val_img_.shape}")
print(f"fixed_val_msk_ shape: {fixed_val_msk_.shape}")
print(f"moving_val_img_ shape: {moving_val_img_.shape}")
print(f"moving_val_msk_ shape: {moving_val_msk_.shape}")


print(f"fixed_train_img_ Range: {fixed_train_img_.max()} {fixed_train_img_.min()}")
print(f"fixed_train_msk_ range: {fixed_train_msk_.max()} {fixed_train_msk_.min()} {np.unique(fixed_train_msk_)}")
print(f"moving_train_img_ Range: {moving_train_img_.max()} {moving_train_img_.min()}")
print(f"moving_train_msk_ range: {moving_train_msk_.max()} {moving_train_msk_.min()} {np.unique(moving_train_msk_)}")

print(f"fixed_val_img_ Range: {fixed_val_img_.max()} {fixed_val_img_.min()}")
print(f"fixed_val_msk_ range: {fixed_val_msk_.max()} {fixed_val_msk_.min()} {np.unique(fixed_val_msk_)}")
print(f"moving_val_img_ Range: {moving_val_img_.max()} {moving_val_img_.min()}")
print(f"moving_val_msk_ range: {moving_val_msk_.max()} {moving_val_msk_.min()} {np.unique(moving_val_msk_)}")


plt.figure("check", (10, 5))

plt.subplot(2, 4, 1)
plt.title("fixed_train_img_")
plt.imshow(fixed_train_img_, cmap="gray")
plt.axis('off')

plt.subplot(2, 4, 2)
plt.title("fixed_train_msk_")
plt.imshow(fixed_train_msk_, cmap="gray")
plt.axis('off')

plt.subplot(2, 4, 3)
plt.title("moving_train_img_")
plt.imshow(moving_train_img_, cmap="gray")
plt.axis('off')

plt.subplot(2, 4, 4)
plt.title("moving_train_msk_")
plt.imshow(moving_train_msk_, cmap="gray")
plt.axis('off')


plt.subplot(2, 4, 5)
plt.title("fixed_val_img_")
plt.imshow(fixed_val_img_, cmap="gray")
plt.axis('off')

plt.subplot(2, 4, 6)
plt.title("fixed_val_msk_")
plt.imshow(fixed_val_msk_, cmap="gray")
plt.axis('off')

plt.subplot(2, 4, 7)
plt.title("moving_val_img_")
plt.imshow(moving_val_img_, cmap="gray")
plt.axis('off')

plt.subplot(2, 4, 8)
plt.title("moving_val_msk_")
plt.imshow(moving_val_msk_, cmap="gray")
plt.axis('off')

plt.show()

In [ ]:
mod = RegUNet(
    spatial_dims=2,
    in_channels=2,
    num_channel_initial=32,
    depth = 5,
    extract_levels=[5],
    out_activation=None,
    out_channels=2,
    out_kernel_initializer="zeros",
    concat_skip=False)

        

inputs = torch.randn((2, 2, img_size, img_size))

y = mod(inputs)
print(y.shape)

import torch, torchinfo
# from torchviz import make_dot, make_dot_from_trace
    
# make_dot(y, params=dict(mod.named_parameters()),
#          show_attrs=False,
#          show_saved=False).render(fileNames, format="png")

torchinfo.summary(mod, input_size=(2, 2, img_size, img_size), depth=100)

In [ ]:
model = mod.to(device)


warp_layer = Warp().to(device)

# image_loss = MSELoss()

image_loss = GlobalMutualInformationLoss()

class SSIMLoss(SSIM):
    def forward(self, x, y):
        return 1. - super().forward(x, y)
    

label_SSIM = SSIMLoss(n_channels=1).to(device) # .cuda() if you need GPU support

label_loss = DiceLoss()

# label_loss = MultiScaleLoss(label_loss, scales=[0, 1, 2, 4, 8, 16])

regularization = BendingEnergyLoss()

optimizer = torch.optim.Adam(model.parameters(), 0.0002)

dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)


# dice_metric = compute_meandice(y_pred, y, include_background=True)

In [ ]:
def make_one_hot(labels, C=2):
    '''
    Converts an integer label torch.autograd.Variable to a one-hot Variable.
    
    Parameters
    ----------
    labels : torch.autograd.Variable of torch.cuda.LongTensor
        N x 1 x H x W, where N is batch size. 
        Each value is an integer representing correct classification.
    C : integer. 
        number of classes in labels.
    
    Returns
    -------
    target : torch.autograd.Variable of torch.cuda.FloatTensor
        N x C x H x W, where C is class number. One-hot encoded.
    '''
    labels = labels.long()
    one_hot = torch.FloatTensor(labels.size(0), C, labels.size(2), labels.size(3)).zero_().to(device)
    target = one_hot.scatter_(1, labels.data, 1)
    
    target = Variable(target)
        
    return target

In [ ]:
class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):  
        super(VariationalEncoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, stride=2, padding=1)
        self.batch2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 3, stride=2, padding=1)  
        self.linear1 = nn.Linear(img_size//8*img_size//8*32, 128)
        self.linear2 = nn.Linear(128, latent_dims)
        self.linear3 = nn.Linear(128, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.to(device) # hack to get sampling on the GPU
        self.N.scale = self.N.scale.to(device)
        self.kl = 0

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.conv1(x))
        x = F.relu(self.batch2(self.conv2(x)))
        x = F.relu(self.conv3(x))
        # print(f'XX = {x.shape}')
        x = torch.flatten(x, start_dim=1)
        # print(f'XX = {x.shape}')
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z      


class Decoder(nn.Module):
    
    def __init__(self, latent_dims):
        super().__init__()

        self.decoder_lin = nn.Sequential(
            nn.Linear(latent_dims, 128),
            nn.ReLU(True),
            nn.Linear(128, img_size//8*img_size//8*32),
            nn.ReLU(True)
        )

        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(32, img_size//8, img_size//8))
        
        self.m = nn.Softmax(dim=1)

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 3, stride=2, padding=1, output_padding=1)
        )
        
    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
#         x = self.m(x)
        return x

class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        x = x.to(device)
        z = self.encoder(x)
#         print(f'Latent dim = {z.shape}')
#         print(f'Latent Value = {z}')
        return self.decoder(z)

### Set the random seed for reproducible results
torch.manual_seed(0)

d = 8

Myo_VAE = VariationalAutoencoder(latent_dims=d)
Myo_VAE.to(device)
Myo_VAE.load_state_dict(torch.load('ACVAE Weights/'+'Myo_VAE_'+ str(img_size) + '_.pth', map_location= device))
Myo_VAE.eval()

# torchinfo.summary(Myo_VAE, input_size=(2, 1, 512, 512), depth=100)

LV_VAE = VariationalAutoencoder(latent_dims=d)
LV_VAE.to(device)
LV_VAE.load_state_dict(torch.load('ACVAE Weights/'+'LV_VAE_'+ str(img_size) + '_.pth', map_location= device))
LV_VAE.eval()

# torchinfo.summary(LV_VAE, input_size=(2, 1, 512, 512), depth=100)

In [ ]:
L2_loss = nn.MSELoss(reduction='mean')

def globalLoss(trueMask, predMask):
    myo_trueMask = trueMask.clone()
    LV_trueMask = trueMask.clone()

    #Thresholding the prediction
    thresholded_predMask = predMask.clone()

    background_class = torch.zeros_like(predMask)
    myo_class = torch.ones_like(predMask)
    lv_class = 2*torch.ones_like(predMask)

    thresholded_predMask_ = torch.where(thresholded_predMask<0.98, background_class, thresholded_predMask)
    thresholded_predMask_ = torch.where(thresholded_predMask_>1.0, lv_class, thresholded_predMask_)
    thresholded_predMask_ = torch.where((thresholded_predMask_<=1.0) & (thresholded_predMask_>=0.98), myo_class, thresholded_predMask_)

    myo_predMask = thresholded_predMask_.clone()
    LV_predMask = thresholded_predMask_.clone()

    myo_predMask[myo_predMask==2] =0 
    myo_trueMask[myo_trueMask==2] =0 

    LV_predMask[LV_predMask==1] =0 
    LV_predMask = LV_predMask/2

    LV_trueMask[LV_trueMask==1] =0 
    LV_trueMask = LV_trueMask/2

    # plt.subplot(241), plt.imshow(myo_predMask[0,:,:,:].reshape(512,512).detach().cpu().squeeze().numpy(), cmap='gist_gray')
    # plt.subplot(242), plt.imshow(myo_trueMask[0,:,:,:].reshape(512,512).detach().cpu().squeeze().numpy(), cmap='gist_gray')
    # plt.subplot(243), plt.imshow(LV_predMask[0,:,:,:].reshape(512,512).detach().cpu().squeeze().numpy(), cmap='gist_gray')
    # plt.subplot(244), plt.imshow(LV_trueMask[0,:,:,:].reshape(512,512).detach().cpu().squeeze().numpy(), cmap='gist_gray')

    attribute_myo_true = Myo_VAE(myo_trueMask)
    attribute_myo_pred = Myo_VAE(myo_predMask)

    myo_L2 = L2_loss(attribute_myo_true, attribute_myo_pred)

    attribute_lv_true = LV_VAE(LV_trueMask)
    attribute_lv_pred = LV_VAE(LV_predMask)

    # print(LV_trueMask.shape)
    # print(attribute_lv_true.shape)

    lv_L2 = L2_loss(attribute_lv_true, attribute_lv_pred)

    # print(myo_L2)
    # print(lv_L2)

    loss_ = myo_L2 + lv_L2

    return loss_

# globalLoss (TTg, TTp)
# globalLoss (TTg, TTm)

In [ ]:
# to create real labels (1s)
def label_real(size):
    data = torch.ones(size, 1)
    return data.to(device)
# to create fake labels (0s)
def label_fake(size):
    data = torch.zeros(size, 1)
    return data.to(device)

# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.n_input = 262144
#         self.main = nn.Sequential(
#             nn.Linear(self.n_input, 1024),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(0.3),

#             # nn.Linear(4096, 2048),
#             # nn.LeakyReLU(0.2),
#             # nn.Dropout(0.3),

#             # nn.Linear(1024, 1024),
#             # nn.LeakyReLU(0.2),
#             # nn.Dropout(0.3),

#             nn.Linear(1024, 512),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(0.4),

#             nn.Linear(512, 256),
#             nn.LeakyReLU(0.2),
#             nn.Dropout(0.4),

#             nn.Linear(256, 1),
#             # nn.Linear(64, 1),
#             nn.Dropout(0.4),
#             nn.Sigmoid(),
#         )
#     def forward(self, x):
#         x = x.view(-1, 262144)
#         return self.main(x)

# Discriminator Model Class Definition
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.main = nn.Sequential(
#             # Block 1: input is (3) x 64 x 64
#             nn.Conv2d(1, 64, 4, 2, 1, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
#             # Block 2: input is (64) x 32 x 32
#             nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(64 * 2),
#             nn.LeakyReLU(0.2, inplace=True),
#             # Block 3: input is (64*2) x 16 x 16
#             nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(64 * 4),
#             nn.LeakyReLU(0.2, inplace=True),
#             # Block 4: input is (64*4) x 8 x 8
#             nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
#             nn.BatchNorm2d(64 * 8),
#             nn.LeakyReLU(0.2, inplace=True),
#             # Block 5: input is (64*8) x 4 x 4
#             nn.Conv2d(64 * 8, 64 * 16, 4, 2, 0, bias=False),
#             nn.BatchNorm2d(64 * 16),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64 * 16, 64 * 16, 4, 2, 0, bias=False),
#             nn.BatchNorm2d(64 * 16),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64 * 16, 64 * 8, 4, 2, 0, bias=False),
#             nn.BatchNorm2d(64 * 8),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64 * 8, 1, 2, 1, 0, bias=False),
#             nn.Flatten(),
#             nn.Sigmoid()
#             # nn.Linear(841, 256),
#             # nn.LeakyReLU(0.2),
#             # nn.Dropout(0.3),
#             # nn.Linear(256, 1),
#             # nn.Sigmoid()
#         )
 
#     def forward(self, input):
#         output = self.main(input)
#         # print(output.shape)
#         return output

Discrim = monai.networks.nets.Discriminator(
    in_shape=(1, img_size, img_size),
    channels=(8, 16, 32, 64, 1),
    strides=(2, 2, 2, 2, 1),
    num_res_units=2,
    kernel_size=3,
    dropout=0.4
)

    
discriminator = Discrim.to(device)
print('\n##### DISCRIMINATOR #####')
print(discriminator)
print('######################')

# loss function
criterion = nn.BCELoss()

# optimizers
# optim_g = optim.Adam(generator.parameters(), lr=0.0002)
optim_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

torchinfo.summary(Discrim, input_size=(2, 1, img_size, img_size), depth=100)

In [ ]:
max_epochs = EP
epoch_loss_values = []
val_interval = 1
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
epoch_val_loss_values = []
epoch_loss_d_values = []
epoch_loss_g_values = []
metric_values = []
best_loss = 1e10
best_dsc = 0
k=1


for epoch in range(max_epochs):
    print("-" * 100)
    print(f"Epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss, step = 0, 0

    epoch_DSC, epoch_L2, epoch_MI, epoch_ddf = 0, 0, 0, 0

    epoch_loss_d, epoch_loss_g=0,0
    
    for fixed_train_img_, fixed_train_msk_, moving_train_img_, moving_train_msk_ in zip(fixed_train_img,
                                                                                    fixed_train_msk, moving_train_img,
                                                                                    moving_train_msk):    
        step += 1
        
        optimizer.zero_grad()
        optim_d.zero_grad()

        fixed_train_img_ = fixed_train_img_.to(device)
        fixed_train_msk_ = fixed_train_msk_.to(device)
        moving_train_img_ = moving_train_img_.to(device)
        moving_train_msk_ = moving_train_msk_.to(device)
        
        ddf_train = model(torch.cat((moving_train_img_, fixed_train_img_), dim=1))
        
        pred_image_train = warp_layer(moving_train_img_, ddf_train)
        pred_mask_train = warp_layer(moving_train_msk_, ddf_train)

        real_label = label_real(fixed_train_img_.size(0))
        fake_label = label_fake(fixed_train_img_.size(0))


        output_real = discriminator(fixed_train_img_)
        loss_real = criterion(output_real, real_label)
        loss_real.backward()

        output_fake = discriminator(pred_image_train.detach())
        loss_fake = criterion(output_fake, fake_label)
        loss_fake.backward()

        optim_d.step()

        loss_d = (loss_real + loss_fake)/2

        epoch_loss_d += loss_d.item()

        output = discriminator(pred_image_train)
        
        loss_g = criterion(output, real_label)

        imageLOSS = image_loss(pred_image_train, fixed_train_img_)
        
        labelDSC = label_loss(fixed_train_msk_, pred_mask_train)

        # labelDSC = label_loss(make_one_hot(pred_mask_train, C=3), make_one_hot(fixed_train_msk_, C=3))

        labelGlobal =  globalLoss(fixed_train_msk_, pred_mask_train)-1
        
        regLOSS  = regularization(ddf_train)

        
        loss = imageLOSS + regLOSS + 2*labelGlobal + 2*labelDSC + 0.001*loss_g
    
        # loss_g.backward(retain_graph=True)
        loss.backward()

        optimizer.step()
        
        epoch_loss += loss.item()

        epoch_loss_g += loss_g.item()
        epoch_DSC += labelDSC.item()
        epoch_L2 += labelGlobal.item()
        epoch_MI += imageLOSS.item()
        epoch_ddf += regLOSS.item()


        
        dice_metric(y_pred = make_one_hot(pred_mask_train, C=3), y=make_one_hot(fixed_train_msk_, C=3))

    metric = dice_metric.aggregate().item()
    dice_metric.reset()
    epoch_loss /= step

    epoch_loss_g /=step
    epoch_loss_d /= step

    epoch_DSC /= step
    epoch_L2 /= step
    epoch_MI /= step
    epoch_ddf /= step

    epoch_loss_values.append(epoch_loss)
    epoch_loss_d_values.append(epoch_loss_d)
    epoch_loss_g_values.append(epoch_loss_g)

    print(f"For epoch: {epoch + 1}, Average train loss: {epoch_loss:.5f} !")
    print(f"For epoch: {epoch + 1}, Average train DSC: {metric:.5f} !\n")

    print(f"For epoch: {epoch + 1}, Average train DSC: {epoch_DSC:.5f} !")
    print(f"For epoch: {epoch + 1}, Average train L2: {epoch_L2:.5f} !")
    print(f"For epoch: {epoch + 1}, Average train MI: {epoch_MI:.5f} !")
    print(f"For epoch: {epoch + 1}, Average train DDF: {epoch_ddf:.5f} !")
    print(f"For epoch: {epoch + 1}, Average train D Loss: {epoch_loss_d:.5f} !")
    print(f"For epoch: {epoch + 1}, Average train G Loss: {epoch_loss_g:.5f} !")
    
    print("-" * 60)
    if (epoch + 1) % val_interval == 0 or epoch == 0:
        model.eval()
        val_loss = 0
        val_met = 0
        step = 0
        with torch.no_grad():
            for fixed_val_img_, fixed_val_msk_, moving_val_img_, moving_val_msk_ in zip(fixed_val_img, fixed_val_msk,
                                                                                        moving_val_img, moving_val_msk):  

                fixed_val_img_ = fixed_val_img_.to(device)
                fixed_val_msk_ = fixed_val_msk_.to(device)
                
                moving_val_img_ = moving_val_img_.to(device)
                moving_val_msk_ = moving_val_msk_.to(device)
                
                
                ddf_val = model(torch.cat((moving_val_img_, fixed_val_img_), dim=1))
                
                pred_image_val = warp_layer(moving_val_img_, ddf_val)
                pred_mask_val = warp_layer(moving_val_msk_, ddf_val)

                real_label = label_real(pred_image_val.size(0))
        
                output = discriminator(pred_image_val)
        
                loss_g = criterion(output, real_label)

                imageLOSS = image_loss(pred_image_val, fixed_val_img_)

                labelDSC = label_loss(fixed_val_msk_, pred_mask_val)

                # labelDSC = label_loss(make_one_hot(pred_mask_val, C=3), make_one_hot(fixed_val_msk_, C=3))

                labelGlobal =  globalLoss(fixed_val_msk_, pred_mask_val)-1

                regLOSS  = regularization(ddf_val)


                loss = imageLOSS + regLOSS + 2*labelGlobal + 2*labelDSC + 0.001*loss_g
                
                
    
                val_loss += loss.item()
                step += 1

                
                dice_metric(y_pred = make_one_hot(pred_mask_val, C=3), y=make_one_hot(fixed_val_msk_, C=3))
                

            val_loss /= step
            epoch_val_loss_values.append(val_loss)
            metric = dice_metric.aggregate().item()
            metric_values.append(metric)
            dice_metric.reset()
            
            if metric > best_dsc:
                best_loss_epoch = epoch + 1
                torch.save(model.state_dict(),
                           os.path.join(root_dir, fileNames+'.pth'))
                print(f"Valid DSC improved from {best_dsc:2.4f} to {metric:2.4f}! Best model is saving as---> {fileNames+'.pth'}")
                best_dsc = metric
                
            print(
                f"\nCurrent mean dice: {metric:.4f} "
                f"\t and \t Current val loss: {val_loss:.4f}\n\n"
                f"Best val DSC: {best_dsc:.4f} "
                f"at epoch number of {best_loss_epoch}")

In [ ]:
# plot and save the generator and discriminator loss
plt.figure("train", (20, 4))
plt.subplot(121)
plt.plot(epoch_loss_values, label='Generator loss')
plt.subplot(122)
plt.plot(epoch_loss_d_values, label='Discriminator Loss')
plt.plot(epoch_loss_g_values, label='Generator Loss')
plt.legend()

In [ ]:
%matplotlib inline
plt.figure("train", (20, 4))
plt.subplot(1, 2, 1)
plt.plot(epoch_loss_values, label = 'training', color='k', lw=2, linestyle='-', marker = 'o')
plt.plot(epoch_val_loss_values, label = 'val', color='b', lw=2, linestyle='-', marker = 'x')
plt.legend(fontsize=20, loc='upper right', ncol=1)
plt.xticks(range(0, max_epochs , 1))
plt.ylabel('Losses', fontsize=20)
plt.xlabel('Epochs', fontsize=20)

plt.subplot(1, 2, 2)
plt.plot(metric_values, label = 'training', color='k', lw=2, linestyle='-', marker = 'o')
plt.ylabel('DSC', fontsize=20)
plt.xlabel('Epochs', fontsize=20)

In [ ]:
def hausdorff (mask1, mask2):
    mask1_class_LV = mask1.copy()
    mask1_class_Myo = mask1.copy()
    
    mask1_class_LV[mask1_class_LV==1] = 0
    mask1_class_Myo[mask1_class_Myo==2] = 0
    
    mask2_class_LV = mask2.copy()
    mask2_class_Myo = mask2.copy()
    
    mask2_class_LV[mask2_class_LV==1] = 0
    mask2_class_Myo[mask2_class_Myo==2] = 0
    
    hausdorff_Myo = (directed_hausdorff(mask1_class_Myo, mask2_class_Myo)[0]+directed_hausdorff(mask2_class_Myo, mask1_class_Myo)[0])/2

    hausdorff_LV = (directed_hausdorff(mask1_class_LV, mask2_class_LV)[0]+directed_hausdorff(mask2_class_LV, mask1_class_LV)[0])/2
    
    return [hausdorff_Myo, hausdorff_LV]

In [ ]:
model.load_state_dict(torch.load(os.path.join(root_dir, fileNames+'.pth')))
model.eval()

metric = monai.metrics.MSEMetric()

before_MSEMetric = []
after_MSEMetric = []

before_compute_meandice = []
after_compute_meandice = []

before_compute_hausdorff_distance = []
after_compute_hausdorff_distance = []
                                                                         

for fixed_val_img_, fixed_val_msk_, moving_val_img_, moving_val_msk_ in zip(fixed_val_img, fixed_val_msk,
                                                                                        moving_val_img, moving_val_msk): 
    
    fixed_val_img_ = fixed_val_img_.to(device)
    fixed_val_msk_ = fixed_val_msk_.to(device)

    moving_val_img_ = moving_val_img_.to(device)
    moving_val_msk_ = moving_val_msk_.to(device)


    ddf_val = model(torch.cat((moving_val_img_, fixed_val_img_), dim=1))

    pred_image_val = warp_layer(moving_val_img_, ddf_val)
    pred_mask_val = warp_layer(moving_val_msk_, ddf_val)
    
    
    before_MSEMetric.extend(metric(moving_val_img_,fixed_val_img_).detach().cpu().numpy())
    
    after_MSEMetric.extend(metric(pred_image_val,fixed_val_img_).detach().cpu().numpy())
    
#     print(metric(pred_image_val,fixed_val_img_).detach().cpu().numpy())

    before_compute_hausdorff_distance.extend((compute_hausdorff_distance(y_pred = make_one_hot(moving_val_msk_, C=3), y=make_one_hot(fixed_val_msk_, C=3), directed=True)).detach().cpu().numpy())
    after_compute_hausdorff_distance.extend((compute_hausdorff_distance(y_pred = make_one_hot(pred_mask_val, C=3), y=make_one_hot(fixed_val_msk_, C=3), directed=True)).detach().cpu().numpy()) 
    
    
    before_compute_meandice.extend((compute_meandice(y_pred = make_one_hot(moving_val_msk_, C=3), y=make_one_hot(fixed_val_msk_, C=3))).detach().cpu().numpy())
    after_compute_meandice.extend((compute_meandice(y_pred = make_one_hot(pred_mask_val, C=3), y=make_one_hot(fixed_val_msk_, C=3))).detach().cpu().numpy()) 
    
    
#     break

print(np.array(before_MSEMetric).shape)
print(np.array(after_MSEMetric).shape)

print(np.array(before_compute_meandice).shape)
print(np.array(after_compute_meandice).shape)

print()

print(f'MSEMetric W/O registration {np.mean(np.array(before_MSEMetric))}')
print(f'MSEMetric W/ registration = {np.mean(np.array(after_MSEMetric))}')

print(f'Mean dice W/O registration = {np.mean(np.array(before_compute_meandice), axis=0)}')
print(f'Mean dice W/ registration = {np.mean(np.array(after_compute_meandice), axis=0)}')

print(f'Hausdorff_distance W/O registration = {np.mean(np.array(before_compute_hausdorff_distance), axis=0)}')
print(f'Hausdorff_distance W/ registration = {np.mean(np.array(after_compute_hausdorff_distance), axis=0)}')

print()

print('----------------For Report Metric +/- Std-----------------------')
print(f'Mean dice W/O registration = {np.mean(np.mean(np.array(before_compute_meandice), axis=0))} + {np.mean(np.std(np.array(before_compute_meandice), axis=0))}')
print(f'Mean dice W/ registration = {np.mean(np.mean(np.array(after_compute_meandice), axis=0))} + {np.mean(np.std(np.array(after_compute_meandice), axis=0))}')

print(f'Hausdorff_distance W/O registration = {np.mean(np.mean(np.array(before_compute_hausdorff_distance), axis=0))} + {np.mean(np.std(np.array(before_compute_hausdorff_distance), axis=0))}')
print(f'Hausdorff_distance W/ registration = {np.mean(np.mean(np.array(after_compute_hausdorff_distance), axis=0))} + {np.mean(np.std(np.array(after_compute_hausdorff_distance), axis=0))}')

df = ({
     'before_compute_hausdorff_distance':np.mean(np.array(before_compute_hausdorff_distance), axis=1),
     'after_compute_hausdorff_distance'   : np.mean(np.array(after_compute_hausdorff_distance), axis=1),
     'before_compute_meandice':np.mean(np.array(before_compute_meandice), axis=1),
     'after_compute_meandice':np.mean(np.array(after_compute_meandice), axis=1),
     'before_MSEMetric': np.mean(np.array(before_MSEMetric), axis=1),
     'after_MSEMetric':np.mean(np.array(after_MSEMetric), axis=1)
})

pd.DataFrame(df).to_csv(fileNames+'.csv')

In [ ]:
model.load_state_dict(torch.load(os.path.join(root_dir, fileNames+'.pth')))
model.eval()
                                          

for fixed_val_img_, fixed_val_msk_, moving_val_img_, moving_val_msk_ in zip(fixed_val_img, fixed_val_msk,
                                                                                        moving_val_img, moving_val_msk): 
    
    fixed_val_img_ = fixed_val_img_.to(device)
    fixed_val_msk_ = fixed_val_msk_.to(device)

    moving_val_img_ = moving_val_img_.to(device)
    moving_val_msk_ = moving_val_msk_.to(device)


    ddf_val = model(torch.cat((moving_val_img_, fixed_val_img_), dim=1))

    pred_image_val = warp_layer(moving_val_img_, ddf_val)
    pred_mask_val = warp_layer(moving_val_msk_, ddf_val)
    
    
    break

fixed_val_img_ = fixed_val_img_.detach().cpu().numpy()[:, 0]
fixed_val_msk_ = fixed_val_msk_.detach().cpu().numpy()[:, 0]

moving_val_img_ = moving_val_img_.detach().cpu().numpy()[:, 0]
moving_val_msk_ = moving_val_msk_.detach().cpu().numpy()[:, 0]

pred_image_val = pred_image_val.detach().cpu().numpy()[:, 0]
pred_mask_val = pred_mask_val.detach().cpu().numpy()[:, 0]

In [ ]:
def colored (pred_img, true_img): 
    bitwise_and = cv2.bitwise_and(pred_img, true_img)
    
    TP = np.stack((np.zeros_like(bitwise_and), bitwise_and, np.zeros_like(bitwise_and)), axis=-1)
    
    FN = np.stack((true_img-bitwise_and,
                   true_img-bitwise_and,
                   np.zeros_like(true_img-bitwise_and)), axis=-1)
    
    FP = np.stack((pred_img-bitwise_and,
                   pred_img-bitwise_and,
                  np.zeros_like(pred_img-bitwise_and)), axis=-1)
    
    return (255*(TP+FN+FP)).astype('uint8')

In [ ]:
%matplotlib inline
batch_size = testBatch
plt.subplots(batch_size, 7, figsize=(12, 15))

for b in range(batch_size):
    # moving image
    plt.subplot(batch_size, 7, b * 7 + 1)
    plt.axis('off')
    plt.title("moving img")
    plt.imshow(moving_val_img_[b], cmap="gray")
    
    # moving label
    plt.subplot(batch_size, 7, b * 7 + 2)
    plt.axis('off')
    plt.title("moving lab")
    plt.imshow(moving_val_msk_[b], cmap="gray")
    
    
    # fixed image
    plt.subplot(batch_size, 7, b * 7 + 3)
    plt.axis('off')
    plt.title("fixed img")
    plt.imshow(fixed_val_img_[b], cmap="gray")
    
    # fixed label
    plt.subplot(batch_size, 7, b * 7 + 4)
    plt.axis('off')
    plt.title("fixed lab")
    plt.imshow(fixed_val_msk_[b], cmap="gray")
    
    
    # warped moving
    plt.subplot(batch_size, 7, b * 7 + 5)
    plt.axis('off')
    plt.title("Pred Img")
    plt.imshow(pred_image_val[b], cmap="gray")


    # warped moving
    plt.subplot(batch_size, 7, b * 7 + 6)
    plt.axis('off')
    plt.title("Before Reg")
    plt.imshow(colored(moving_val_msk_[b], fixed_val_msk_[b]))
    
    
    # warped moving
    plt.subplot(batch_size, 7, b * 7 + 7)
    plt.axis('off')
    plt.title("After Reg")
    plt.imshow(colored(pred_mask_val[b], fixed_val_msk_[b]))
    
plt.axis('off')
plt.show()